# Mobile Get Input Notebook - Phase 0: Setup

**Phase 0**: Initialize environment, configure parameters, and prepare elevation data.

This notebook sets up everything needed for the subsequent phases:
- Phase 1: Data Preparation (download/cache land cover)
- Phase 2: Batch Point Generation
- Phase 3: Batch Data Extraction
- Phase 4: Post-processing & Export

## Workflow
1. **Imports**: Load required libraries and modules
2. **Path Setup**: Detect project root and create data directories
3. **Configuration**: Define transmitter location and processing parameters
4. **Transmitter**: Create transmitter object from configuration
5. **Elevation Seed**: Pre-download elevation data (one-time operation)

## Output
- Ready-to-use environment with CONFIG, tx, and elevation data cached
- All subsequent phases can reuse this setup

## Imports

In [1]:
import os
import math
import sys
import time
from pathlib import Path

# Find and add project root to path FIRST (before importing config_sentinel_hub)
possible_roots = [
    Path.cwd(),
    Path.cwd().parent,
    Path.cwd().parent.parent,
]

project_root = None
for root in possible_roots:
    if (root / 'src' / 'mst_gis').exists() or (root / 'config_sentinel_hub.py').exists():
        project_root = root
        break

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Now we can import config_sentinel_hub
import requests
from config_sentinel_hub import (
    SH_CLIENT_ID, SH_CLIENT_SECRET,
    TOKEN_URL, PROCESS_URL, COLLECTION_ID,
)

# Data processing imports
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, box
from dataclasses import dataclass
from typing import Iterable, Union, List, Tuple

# GIS and raster imports
import rasterio
import rasterio.transform
from rasterio.transform import rowcol
from rasterio.io import MemoryFile
import srtm

# Additional imports for all phases
import ast
import csv
import json
import matplotlib.pyplot as plt

# P.1812 propagation model
import Py1812.P1812

# Define all data paths (used by later cells)
profiles_dir = project_root / 'data' / 'input' / 'profiles'
api_data_dir = project_root / 'data' / 'intermediate' / 'api_data'
workflow_dir = project_root / 'data' / 'intermediate' / 'workflow'
reference_dir = project_root / 'data' / 'input' / 'reference'
output_dir = project_root / 'data' / 'output' / 'spreadsheets'
geojson_dir = project_root / 'data' / 'output' / 'geojson'
zones_geojson_path = reference_dir / 'zones_map_BR.json'

print("✓ All imports successful")

/Users/oz/Documents/mst_gis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✓ All imports successful


## Path Setup

In [2]:
# Create all required data directories
profiles_dir.mkdir(parents=True, exist_ok=True)
api_data_dir.mkdir(parents=True, exist_ok=True)
workflow_dir.mkdir(parents=True, exist_ok=True)
reference_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)
geojson_dir.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print('✓ All data directories created')
print(f'  profiles: {profiles_dir}')
print(f'  api_data: {api_data_dir}')
print(f'  reference: {reference_dir}')
print(f'  output: {output_dir}')

Project root: /Users/oz/Documents/mst_gis
✓ All data directories created
  profiles: /Users/oz/Documents/mst_gis/data/input/profiles
  api_data: /Users/oz/Documents/mst_gis/data/intermediate/api_data
  reference: /Users/oz/Documents/mst_gis/data/input/reference
  output: /Users/oz/Documents/mst_gis/data/output/spreadsheets


## Configuration

In [3]:
import json
from pathlib import Path

# Load configuration from config_example.json (single source of truth)
config_path = project_root / 'config_example.json'
if not config_path.exists():
    raise FileNotFoundError(f'Config file not found: {config_path}')

with open(config_path, 'r') as f:
    CONFIG = json.load(f)

print(f'✓ Loaded configuration from {config_path.name}')

# Derived values
tx_lon = CONFIG['TRANSMITTER']['longitude']
tx_lat = CONFIG['TRANSMITTER']['latitude']
max_distance_km = CONFIG['RECEIVER_GENERATION']['max_distance_km']
n_points = int(max_distance_km * 1000 / CONFIG['RECEIVER_GENERATION']['sampling_resolution'])
azimuths = list(range(0, 360, CONFIG['RECEIVER_GENERATION']['azimuth_step']))
distances = np.arange(
    CONFIG['RECEIVER_GENERATION']['distance_step'],
    max_distance_km + CONFIG['RECEIVER_GENERATION']['distance_step'],
    CONFIG['RECEIVER_GENERATION']['distance_step']
)

print(f"Transmitter: ({tx_lat}, {tx_lon})")
print(f"Azimuths: {len(azimuths)} | Profile points: {n_points}")
print(f"Frequency: {CONFIG['P1812']['frequency_ghz']} GHz | Polarization: {CONFIG['P1812']['polarization']}")

✓ Loaded configuration from config_example.json
Transmitter: (9.345, -13.40694)
Azimuths: 36 | Profile points: 366
Frequency: 0.9 GHz | Polarization: 1


## Transmitter Definition

In [4]:
@dataclass
class Transmitter:
    tx_id: str
    lon: float
    lat: float
    htg: float
    f: float
    pol: int
    p: float
    hrg: float

tx = Transmitter(
    tx_id=CONFIG['TRANSMITTER']['tx_id'],
    lon=CONFIG['TRANSMITTER']['longitude'],
    lat=CONFIG['TRANSMITTER']['latitude'],
    htg=CONFIG['TRANSMITTER']['antenna_height_tx'],
    f=CONFIG['P1812']['frequency_ghz'],
    pol=CONFIG['P1812']['polarization'],
    p=CONFIG['P1812']['time_percentage'],
    hrg=CONFIG['TRANSMITTER']['antenna_height_rx'],
)
print(f"\n✓ Transmitter created:")
print(f"  {tx}")


✓ Transmitter created:
  Transmitter(tx_id='TX_0001', lon=-13.40694, lat=9.345, htg=57, f=0.9, pol=1, p=50, hrg=10)


## Elevation Data Preparation

Seed elevation data once (required for all subsequent phases).

In [5]:
# KEY OPTIMIZATION: Pre-load SRTM elevation data as array
# This downloads the tile and caches it for use in all phases
print("\nInitializing SRTM elevation data...")
init_start = time.time()

try:
    # Set SRTM cache to project directory
    from mst_gis.propagation.profile_extraction import set_srtm_cache_dir, _get_srtm_data
    srtm_cache = project_root / "data" / "intermediate" / "elevation_cache"
    set_srtm_cache_dir(str(srtm_cache))
    
    # Initialize SRTM data handler
    srtm_data = _get_srtm_data()
    
    # Pre-download and cache tile by querying transmitter location
    print(f"  Downloading SRTM1 tile for TX area ({tx_lat}, {tx_lon})...")
    tx_elev = srtm_data.get_elevation(tx_lat, tx_lon)
    
    init_time = time.time() - init_start
    print(f"✓ SRTM elevation data ready ({init_time:.2f}s)")
    print(f"  TX elevation: {tx_elev}m")
    print(f"  Cache location: {srtm_cache}")
    
    # Load the cached HGT tile into memory as array for Phase 3
    import rasterio
    import glob
    hgt_files = glob.glob(str(srtm_cache / "*.hgt"))
    if hgt_files:
        hgt_path = hgt_files[0]
        print(f"\n  Loading HGT tile into memory for Phase 3...")
        with rasterio.open(hgt_path) as dem_src:
            dem_band_data = dem_src.read(1)
            dem_transform = dem_src.transform
        print(f"  ✓ HGT loaded: {dem_band_data.shape} array, dtype={dem_band_data.dtype}")
    else:
        dem_band_data = None
        dem_transform = None
        print("  ⚠ No HGT files found in cache")
    
except Exception as e:
    import traceback
    print(f"✗ Error initializing SRTM: {e}")
    traceback.print_exc()
    srtm_data = None
    dem_band_data = None
    dem_transform = None


Initializing SRTM elevation data...
✓ SRTM elevation data ready (0.01s)
  TX elevation: 13m
  Cache location: /Users/oz/Documents/mst_gis/data/intermediate/elevation_cache

  Loading HGT tile into memory for Phase 3...
  ✓ HGT loaded: (1201, 1201) array, dtype=int16


## Summary

**Phase 0 Complete**:
- ✓ All imports loaded
- ✓ Project paths created
- ✓ Configuration defined (CONFIG dictionary)
- ✓ Transmitter object created
- ✓ Elevation data cached locally

**Output**: Ready-to-use environment with all setup complete.

**Duration**: ~{seed_time:.2f}s (mainly for elevation.seed())

**Next**: Proceed to Phase 1 (Data Preparation)